In [1]:
import csv

In [2]:
!pip3 install neo4j

In [3]:
from neo4j import GraphDatabase

URI = "neo4j://localhost:7687"
AUTH = ('neo4j', 'neo4j1234')

In [4]:
#Top Country has the highest airports
#MATCH (p:Airport)
#WITH p.country AS Country, COUNT(distinct p) AS airportCount
#ORDER BY airportCount DESC
#LIMIT 10
#RETURN Country, airportCount

def read_TopCountry_w_HighestAirports():
    ll = {}
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        records, summary, keys = driver.execute_query(
            """ 
            MATCH (p:Airport)
            RETURN p.airport_id as Airport_ID, p.country AS Country
            """,
            database_="neo4j", )
        
        for record in records: 
            #print(record.data())
            airport_id = record.data()['Airport_ID']
            country = record.data()['Country']

            if country not in ll.keys():
                ll[country] = [airport_id]
            elif airport_id not in ll[country]:
                ll[country].append(id)   
    top_country = max(ll, key=lambda x: len(ll[x]))
    highest_airports = len(ll[top_country])
    print(f"{top_country}: {highest_airports}")
         
    driver.close() 


In [5]:
read_TopCountry_w_HighestAirports()

United States: 1251


In [31]:
#top k cities with most airlines
#MATCH (p:Airport)-[r:Route]->(p2:Airport)
#WITH p.city AS City, COLLECT(DISTINCT r.airline_name) AS Airlines
#WITH City, Airlines, SIZE(Airlines) AS numofAirlines
#ORDER BY numofAirlines DESC
#LIMIT 10
#RETURN City, Airlines, numofAirlines*/

def read_TopKCities_w_MostAirlines():
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        records, summary, keys = driver.execute_query(
            """ 
            MATCH (p:Airport)-[r:Route]->(p2:Airport)
            WITH p.city AS City, COLLECT(DISTINCT r.airline_name) AS Airlines
            WITH City, Airlines, SIZE(Airlines) AS numofAirlines
            ORDER BY numofAirlines DESC
            RETURN City, Airlines, numofAirlines
            """,
            database_="neo4j", )
        for record in records: 
            city = record.data()['City']
            airline = record.data()['Airlines']
            numofAirlines = record.data()["numofAirlines"]
            row = [city, numofAirlines]
            print(row)
    driver.close() 

In [32]:
read_TopKCities_w_MostAirlines()

['Los Angeles', 17]
['San Diego', 16]
['Las Vegas', 16]
['New York', 15]
['Boston', 14]
['Phoenix', 14]
['Fort Lauderdale', 14]
['Chicago', 14]
['Orlando', 14]
['San Francisco', 13]
['Washington', 13]
['Seattle', 13]
['Dallas-Fort Worth', 13]
['Portland', 13]
['Minneapolis', 13]
['Detroit', 12]
['Fort Myers', 12]
['New Orleans', 12]
['Vancouver', 11]
['Houston', 11]
['Tampa', 11]
['Atlanta', 11]
['Denver', 11]
['Edmonton', 10]
['Ottawa', 10]
['Newark', 10]
['Austin', 10]
['Philadelphia', 10]
['Montreal', 9]
['Oakland', 9]
['Kansas City', 9]
['Memphis', 9]
['Cincinnati', 9]
['Rochester', 9]
['Raleigh-durham', 9]
['St. Louis', 9]
['Nashville', 9]
['Honolulu', 9]
['San Jose', 9]
['Charleston', 9]
['Baltimore', 9]
['Charlotte', 9]
['Halifax', 8]
['Toronto', 8]
['Calgary', 8]
['Kahului', 8]
['Cleveland', 8]
['Myrtle Beach', 8]
['Salt Lake City', 8]
['Harrisburg', 8]
['Pittsburgh', 8]
['Indianapolis', 8]
['Burbank', 8]
['Little Rock', 8]
['Columbus', 8]
['Milwaukee', 8]
['Des Moines', 8]
['S

In [27]:
#top k cities with most airlines
#MATCH (p:Airport)-[r:Route]->(p2:Airport)
#WITH p.city AS City, COLLECT(DISTINCT r.airline_name) AS Airlines
#WITH City, Airlines, SIZE(Airlines) AS numofAirlines
#ORDER BY numofAirlines DESC
#LIMIT 10
#RETURN City, Airlines, numofAirlines*/
def read_TopKCities_w_MostAirlines2():
    city_airlines = {}
    airport_to_city = {}

    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        records, summary, keys = driver.execute_query(
            """ 
            MATCH (p:Airport)
            RETURN p.airport_id as Airport_ID, p.city AS City
            """,
            database_="neo4j", )
        for record in records: 
            #print(record.data())
            airport_id = record.data()['Airport_ID']
            city = record.data()['City']
            airport_to_city[airport_id] = city

        records, summary, keys = driver.execute_query(
            """ 
            MATCH (p:Airport)-[r:Route]->(p2:Airport)
            RETURN r.airline_name as Airline_Name, p.airport_id as Source_airport_id, p2.airport_id as Dest_airport_id
            """,
            database_="neo4j", )
        for record in records: 
            #print(record.data())
            airline_name = record.data()['Airline_Name']
            source_airport_id = record.data()['Source_airport_id']
            dest_airport_id = record.data()['Dest_airport_id']

            source_city = airport_to_city.get(source_airport_id, "Unknown")
            dest_city = airport_to_city.get(dest_airport_id, "Unknown")

            if source_city != "Unknown":
                if source_city not in city_airlines:
                    city_airlines[source_city] = set()
                city_airlines[source_city].add(airline_name)

            if dest_city != "Unknown": 
                if dest_city not in city_airlines:
                    city_airlines[dest_city] = set()
                city_airlines[dest_city].add(airline_name)
    sorted_cities = sorted(city_airlines.items(), key=lambda item: len(item[1]), reverse=True)
    #print(sorted_cities)
    for city, airlines in sorted_cities:
        airlines = list(airlines)
        sorted_airlines = sorted(airlines)
        print(f"{city}, {sorted_airlines}, {len(sorted_airlines)}")

    driver.close()

In [28]:
read_TopKCities_w_MostAirlines2()

Los Angeles, ['4B', 'AA', 'AC', 'B6', 'DL', 'F9', 'FL', 'G4', 'HA', 'NK', 'SY', 'UA', 'US', 'VX', 'WN', 'WS', 'ZK'], 17
Las Vegas, ['4B', 'AA', 'AC', 'B6', 'DL', 'F9', 'FL', 'G4', 'HA', 'NK', 'SY', 'UA', 'US', 'VX', 'WN', 'WS'], 16
San Diego, ['AA', 'AC', 'B6', 'DL', 'F9', 'FL', 'G4', 'HA', 'K5', 'NK', 'SY', 'UA', 'US', 'VX', 'WN', 'WS'], 16
New York, ['9E', 'AA', 'AC', 'B6', 'DL', 'F9', 'FL', 'HA', 'NK', 'SY', 'UA', 'US', 'VX', 'WN', 'WS'], 15
Phoenix, ['AA', 'AC', 'B6', 'DL', 'F9', 'FL', 'HA', 'NK', 'SY', 'UA', 'US', 'WN', 'WS', 'ZK'], 14
Chicago, ['AA', 'AC', 'B6', 'DL', 'F9', 'FL', 'NK', 'PD', 'SY', 'UA', 'US', 'VX', 'WN', 'WS'], 14
Orlando, ['AA', 'AC', 'B6', 'DL', 'F9', 'FL', 'NK', 'SY', 'TS', 'UA', 'US', 'VX', 'WN', 'WS'], 14
Boston, ['9K', 'AA', 'AC', 'B6', 'DL', 'FL', 'KS', 'NK', 'PD', 'SY', 'UA', 'US', 'VX', 'WN'], 14
Fort Lauderdale, ['AA', 'AC', 'B6', 'DL', 'F9', 'FL', 'G4', 'NK', 'TS', 'UA', 'US', 'VX', 'WN', 'WS'], 14
San Francisco, ['AA', 'AC', 'B6', 'DL', 'F9', 'FL', 'H

In [29]:
import psutil

In [30]:
# Check RAM usage
ram_usage = psutil.virtual_memory()
print(f"Total RAM: {ram_usage.total / (1024 ** 3):.2f} GB")
print(f"Available RAM: {ram_usage.available / (1024 ** 3):.2f} GB")
print(f"Used RAM: {ram_usage.used / (1024 ** 3):.2f} GB")
print(f"RAM Usage (%): {ram_usage.percent}%")

# Check CPU usage
cpu_usage = psutil.cpu_percent(interval=1, percpu=True)
print(f"CPU Usage (%): {cpu_usage}")

Total RAM: 16.00 GB
Available RAM: 3.00 GB
Used RAM: 6.24 GB
RAM Usage (%): 81.2%
CPU Usage (%): [74.3, 74.0, 35.0, 23.0, 15.0, 27.0, 12.0, 7.0]
